<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://drive.google.com/uc?export=view&id=1LqkEgbpZj8A99Y9T59eBp9fEIZbpg6P2" alt="Snowflake Snowpark Classroom" style="width: 800px">
</div>

# Classroom 1.2 - Accessing Data Using Snowflake Snowpark Sesson 

In this notebook, you will learn how to interract with the data using Session Methods and accessing them inside dataframes

## Learning Objectives

By the end of this classroom, you should be able to:
- Get understanding on Snowpark Session Methods to interract with data
- Creating dataframes from the snowflake data
- Understanding misc Snowpark Session methods along with Observability

In [ ]:
# Lets get started with Snowpark
from snowflake.snowpark import Session
from assets.config import connection_builder
session = connection_builder()

## Section 1 - Understanding Different Ways to Read Data Using Snowpark Session Methods

There are different ways you can interract with snowflake to fetch the data from the tables or using a SQL syntax. The below mentioned are ways to read the data:

|Snowpark Method|Description|
|--|--|
|Session.table()|Returns a Table that points the specified table|
|Session.sql()|Returns a new DataFrame representing the results of a SQL query. You can use this method to execute a SQL statement|
|Session.call()|Calls a stored procedure by name|
|Session.table_function()|Creates a new DataFrame from the given snowflake SQL table function|


In [4]:
print('----------------- Output From Session.table() Method -----------------')
session.table('SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY').limit(100).show()

print('----------------- Output From Session.sql() Method -----------------')
session.sql(f"SHOW DATABASES IN ACCOUNT").show()

print('----------------- Output From Session.call() Method -----------------')

import snowflake.snowpark
from snowflake.snowpark.functions import sproc
session.add_packages('snowflake-snowpark-python')

# creating a sample sproc for the notebook
session.sql('CREATE TABLE IF NOT EXISTS Q_HISTORY AS SELECT * FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY LIMIT 100').collect()
@sproc(session=session,name='sf_copy_function',replace=True)
def copy_function(session: snowflake.snowpark.Session, src_table:str,tgt_table:str, count:int )->str:
    session.table(src_table).limit(count).write.save_as_table(tgt_table)
    return 'success'
session.table('Q_HISTORY').drop_table()

session.call('sf_copy_function','SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY','Q_HISTORY',10)

print('----------------- Output From Session.table_function() Method -----------------')
session.table_function('information_schema.query_history').show()

----------------- Output From Session.table() Method -----------------
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Section 2 - Creating Dataframes for Query Results

We have 2 major methods to create dataframes using Snowflake Snowpark API. 

| Snowpark Method | Description |
|--|--|
|Session.createDataFrame | Creates a new DataFrame containing the specified values from the local data|
| Session.create_dataframe | Creates a new DataFrame containing the specified values from the local data|


### 2(a) Understanding Schemas and Datatypes

While creating the dataframes, you should be aware of how to build the dataframe schema and define the data types for each fields.

|Class| Description|
|--|--|
| StructType | The snowflake.snowpark.types.StructType Represents a table schema. It contains StructField for each column |
| StructField | The snowflake.snowpark.types.StructField Represents the content of StructField. It returns the column name |

You can get all the list of datatypes on the official Developer Guide - [Snowpark Python API - Datatypes](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/types)

In [5]:
from snowflake.snowpark.types import (
    StructType,
    StructField,
    IntegerType,
    FloatType,
    StringType,
)

schema = StructType([StructField("name", StringType()), StructField('age',IntegerType()), StructField('compa_ratio',FloatType())])

dataframe1 = session.createDataFrame([['Divyansh',25,2.2],['Ashok',29,1.6],['Piyush',35,2.5]],schema=schema)
dataframe1.show()

dataframe1 = session.create_dataframe([['Divyansh',25,2.2],['Ashok',29,1.6],['Piyush',35,2.5]],schema=schema)
dataframe1.show()


------------------------------------
|"NAME"    |"AGE"  |"COMPA_RATIO"  |
------------------------------------
|Divyansh  |25     |2.2            |
|Ashok     |29     |1.6            |
|Piyush    |35     |2.5            |
------------------------------------

------------------------------------
|"NAME"    |"AGE"  |"COMPA_RATIO"  |
------------------------------------
|Divyansh  |25     |2.2            |
|Ashok     |29     |1.6            |
|Piyush    |35     |2.5            |
------------------------------------



## Section 3 - Miscellaneous Methods for Accessing and Generating the Data Using Snowpark Python API

In [6]:
# Flattening the JSON Data Using snowpark.Session.flatten
from snowflake.snowpark.functions import lit, parse_json
json_data = '''
{'widget': {
    'debug': 'on',
    'window': {
        'title': 'Sample Konfabulator Widget',
        'name': 'main_window',
        'width': 500,
        'height': 500
    },
    'image': { 
        'src': 'Images/Sun.png',
        'name': 'sun1',
        'hOffset': 250,
        'vOffset': 250,
        'alignment': 'center'
    },
    'text': {
        'data': 'Click Here',
        'size': 36,
        'style': 'bold',
        'name': 'text1',
        'hOffset': 250,
        'vOffset': 100,
        'alignment': 'center',
        'onMouseUp': 'sun1.opacity = (sun1.opacity / 100) * 90;'
    }
}}    
'''

session.flatten(parse_json(lit(json_data)),'widget.image',False,False,'BOTH').show()

Session.flatten() is deprecated since 0.7.0. Use `Session.table_function()` instead.


--------------------------------------------------------------------------------------------------------
|"SEQ"  |"KEY"      |"PATH"                  |"INDEX"  |"VALUE"           |"THIS"                      |
--------------------------------------------------------------------------------------------------------
|1      |alignment  |widget.image.alignment  |NULL     |"center"          |{                           |
|       |           |                        |         |                  |  "alignment": "center",    |
|       |           |                        |         |                  |  "hOffset": 250,           |
|       |           |                        |         |                  |  "name": "sun1",           |
|       |           |                        |         |                  |  "src": "Images/Sun.png",  |
|       |           |                        |         |                  |  "vOffset": 250            |
|       |           |                        |         

In [7]:
# Creating a new DataFrame using the Generator table function

from snowflake.snowpark.functions import seq8, uniform

session.generator(seq8(1).as_('Sequence'),uniform(1,100,2).as_('Uniform'),timelimit=2).show()

--------------------------
|"SEQUENCE"  |"UNIFORM"  |
--------------------------
|0           |24         |
|1           |24         |
|2           |24         |
|3           |24         |
|4           |24         |
|5           |24         |
|6           |24         |
|7           |24         |
|8           |24         |
|9           |24         |
--------------------------



In [8]:
# Create an instance of QueryHistory as a context manager to record queries that are pushed down to the Snowflake database. [Observability]

with session.query_history() as qh:
    session.sql('show databases').show()
    session.table('Q_HISTORY').drop_table()
    session.call('sf_copy_function','SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY','Q_HISTORY',10)

qh.queries

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"                      |"is_default"  |"is_current"  |"origin"                                            |"owner"       |"comment"                                          |"options"  |"retention_time"  |"kind"             |"budget"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2023-10-08 00:31:14.717000-07:00  |HEALTH_CHECK_FOR_SNOWFLAKE  |N             |N             |                                                    |ACCOUNTADMIN  |   

[QueryRecord(query_id='01af8218-3200-e4e2-0004-ebfe0003f182', sql_text='show databases'),
 QueryRecord(query_id='01af8218-3200-e4a1-0004-ebfe0004116e', sql_text='drop table Q_HISTORY'),
 QueryRecord(query_id='01af8218-3200-e4a3-0004-ebfe0003b1f6', sql_text="CALL sf_copy_function('SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY', 'Q_HISTORY', 10 :: INT)")]

In [9]:
# Creates a new DataFrame from a range of numbers. The resulting DataFrame has single column named ID, containing elements in a range from start to end (exclusive) with the step value step.
session.range(101,201,3).show()

--------
|"ID"  |
--------
|101   |
|104   |
|107   |
|110   |
|113   |
|116   |
|119   |
|122   |
|125   |
|128   |
--------

